In [1]:
import torch

In [6]:
points = torch.tensor([[4, 1], [5,3], [2,1.0]])
points, points.storage()

(tensor([[4., 1.],
         [5., 3.],
         [2., 1.]]),
  4.0
  1.0
  5.0
  3.0
  2.0
  1.0
 [torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6])

points will report that its size is (3,2), but the storage is just a contiguous array.

We can index into a storage:

In [7]:
points.storage()[0]

4.0

and modifying the storage unsurprisingly modifies the tensor's view of it

In [9]:
points.storage()[0] = -99.0
points

tensor([[-99.,   1.],
        [  5.,   3.],
        [  2.,   1.]])

as we've seen before, methods which end in underscore modify the underlying storage; all others return a copy

In [10]:
points.zero_()
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

## Tensor Metadata

tensors are defined by three pieces of data:
- **size** is a tuple indicating how many elements across each dimension the tensor represents
- **offset** is the index in the storage corresponding to the first element in the tensor
- **stride** is the number of elements in the storage that need to be skipped over to obtain the next element along each dimension

The second point in the tensor has offset 2 (since it's the second _row_ and needs to skip the first two values) and size is `[2]` because it's a one-dimensional tensor of size 2

In [11]:
points[1].storage_offset(), points[1].size()

(2, torch.Size([2]))

`stride` is a tuple indicating the number of elements that must be skipped when the index is increased by 1 in each dimension

In [12]:
points.stride()

(2, 1)

in a 2d tensor, accessing element `(i,j)` is:

`storage_offset + (stride[0]*i) + (stride[1]*j)`

because two tensors may share storage, modifying a value in one will cause it to be reflected in the other

In [15]:
a = torch.rand(3, 3)
print(a)
b = a[1:, 1:]
b[0,0] = .9999
print(a)

tensor([[0.8709, 0.9848, 0.0072],
        [0.1993, 0.4801, 0.8245],
        [0.5707, 0.4890, 0.2560]])
tensor([[0.8709, 0.9848, 0.0072],
        [0.1993, 0.9999, 0.8245],
        [0.5707, 0.4890, 0.2560]])


In [19]:
a = torch.ones(3, 4, 5)
a_t = a.transpose(0, 2)
a.shape, a_t.shape

(torch.Size([3, 4, 5]), torch.Size([5, 4, 3]))

In [20]:
a, a_t

(tensor([[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]]),
 tensor([[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],
 
         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],
 
         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],
 
         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],
 
         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]))

A transpose of a contiguous tensor is not contiguous

In [21]:
a.is_contiguous(), a_t.is_contiguous()

(True, False)

In [23]:
p = torch.tensor([[4,1],[5,3],[2,1.0]])
p_t = p.t()
p_t, p_t.storage(), p_t.stride(), p_t.is_contiguous()

(tensor([[4., 5., 2.],
         [1., 3., 1.]]),
  4.0
  1.0
  5.0
  3.0
  2.0
  1.0
 [torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6],
 (1, 2),
 False)

In [24]:
p_t_c = p_t.contiguous()
p_t_c, p_t_c.storage(), p_t_c.stride()

(tensor([[4., 5., 2.],
         [1., 3., 1.]]),
  4.0
  5.0
  2.0
  1.0
  3.0
  1.0
 [torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6],
 (3, 1))